## Initial notebook 
to test methods to load/process audio data using torch and train models

In [1]:
# torch and torchaudio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchaudio

# other usefull imports
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

### Download and load dataset

In [4]:
ds = torchaudio.datasets.DR_VCTK(
    root = "C:/Users/Christian/Documents/GitHub/voice2voice/datasets/", 
    subset = 'train', 
    download = True, 
    url = 'https://datashare.ed.ac.uk/bitstream/handle/10283/3038/DR-VCTK.zip',
)

dl = DataLoader(
    dataset=ds,
    batch_size=1,
    shuffle=False
)
data_iter = iter(dl)

In [13]:
waveform_clean, sample_rate_clean, waveform_noisy, sample_rate_noisy, speaker_id, utterance_id, source, channel_id = ds[0]

In [11]:
len(ds)

11572